In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any
import time

# Statistical modeling
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All imports loaded successfully!")

In [ ]:
# Cell 2: Data Loading Function

import sys
from pathlib import Path

# Add src to path and import build_training_set function
src_path = Path("src").resolve()
if src_path.exists():
    sys.path.insert(0, str(src_path))
from utils.build_training_set import build_training_set

def load_training_data(train_start, train_end, run_date):
    """Load training data using build_training_set function"""
    training_set = build_training_set(
        train_start=train_start,
        train_end=train_end, 
        run_date=run_date
    )
    
    if training_set is not None:
        training_data = training_set.copy()
        if 'target_datetime' in training_data.columns:
            training_data = training_data.set_index('target_datetime')
        return training_data
    else:
        raise ValueError("Failed to build training set")

def get_rolling_training_data(origin_date, window_days=73):
    """Get training data for rolling window experiment"""
    origin = pd.to_datetime(origin_date, utc=True)
    train_start = origin - timedelta(days=window_days)
    train_end = origin - timedelta(hours=1)
    run_date = origin
    
    return load_training_data(
        train_start=train_start.strftime('%Y-%m-%d %H:%M:%S'),
        train_end=train_end.strftime('%Y-%m-%d %H:%M:%S'),
        run_date=run_date.strftime('%Y-%m-%d %H:%M:%S')
    )

print("✅ Data loading functions ready!")

In [ ]:
# Cell 3: Ultra-Fast SARIMAX Configuration

# Model configuration - optimized for speed
SARIMA_ORDER = (1, 0, 0)  # Simple AR(1) for speed
SEASONAL_ORDER = (0, 0, 0, 0)  # No seasonality for speed
TRAINING_WINDOW_DAYS = 73
FORECAST_HORIZON_DAYS = 7
REFIT_FREQUENCY = 7

# All your exogenous variables
EXOG_VARIABLES = [
    'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 
    'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst',
    'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 'hour_cos', 'is_weekend',
    'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos'
]

def fit_fast_models(price_series, exog_data=None):
    """Fit models with speed optimizations"""
    models = {'naive': None, 'sarima': None, 'sarimax': None}
    
    # SARIMA with reduced iterations
    try:
        sarima_model = SARIMAX(price_series, order=SARIMA_ORDER, seasonal_order=SEASONAL_ORDER,
                              enforce_stationarity=False, enforce_invertibility=False)
        models['sarima'] = sarima_model.fit(disp=False, maxiter=50, method='lbfgs')
    except:
        pass
    
    # SARIMAX with reduced iterations
    if exog_data is not None:
        try:
            sarimax_model = SARIMAX(price_series, exog=exog_data, order=SARIMA_ORDER, 
                                   seasonal_order=SEASONAL_ORDER, enforce_stationarity=False, 
                                   enforce_invertibility=False)
            models['sarimax'] = sarimax_model.fit(disp=False, maxiter=30, method='lbfgs')
        except:
            pass
    
    return models

def generate_7day_forecasts(models, last_price, exog_forecast=None):
    """Generate 7-day forecasts from fitted models"""
    forecasts = {
        'naive': [last_price] * 7,
        'sarima': [last_price] * 7,
        'sarimax': [last_price] * 7
    }
    
    # SARIMA forecast
    if models['sarima']:
        try:
            forecasts['sarima'] = models['sarima'].forecast(steps=7).tolist()
        except:
            pass
    
    # SARIMAX forecast
    if models['sarimax'] and exog_forecast is not None:
        try:
            forecasts['sarimax'] = models['sarimax'].forecast(steps=7, exog=exog_forecast).tolist()
        except:
            pass
    
    return forecasts

print("✅ Ultra-fast SARIMAX configuration ready!")

In [ ]:
# Cell 3: Main Rolling Forecast Class

class RollingForecastExperiment:
    """
    Implements 30-day rolling forecast experiment for electricity price forecasting
    """
    
    def __init__(self, training_data: pd.DataFrame, 
                 sarima_order: Tuple[int, int, int] = (1, 1, 1),
                 seasonal_order: Tuple[int, int, int, int] = (1, 1, 1, 24),
                 exog_variables: List[str] = None):
        """
        Initialize the rolling forecast experiment
        
        Args:
            training_data: DataFrame with Price and exogenous variables
            sarima_order: (p, d, q) parameters for SARIMA
            seasonal_order: (P, D, Q, s) parameters for seasonal component
            exog_variables: List of exogenous variable column names
        """
        self.data = training_data.copy()
        self.sarima_order = sarima_order
        self.seasonal_order = seasonal_order
        self.exog_variables = exog_variables or []
        
        # Results storage
        self.results = {
            'dates': [],
            'actual_prices': [],
            'naive_forecasts': [],
            'sarima_forecasts': [],
            'sarimax_forecasts': [],
            'naive_rmse': [],
            'sarima_rmse': [],
            'sarimax_rmse': []
        }
        
        # Model storage for analysis
        self.fitted_models = {
            'sarima': [],
            'sarimax': []
        }
        
        print(f"🔧 Experiment initialized:")
        print(f"   - SARIMA order: {sarima_order}")
        print(f"   - Seasonal order: {seasonal_order}")
        print(f"   - Exogenous variables: {len(self.exog_variables)}")
    
    
    def get_actual_data_end_date(self) -> pd.Timestamp:
        """
        Find the last date with actual price data (not forecasted)
        Assumes forecasted data has been appended to the end
        """
        # This is a heuristic - you might need to adjust based on your data structure
        # Look for the last non-interpolated or non-repeated price pattern
        price_series = self.data['Price']
        
        # Simple approach: assume actual data is complete up to a certain point
        # You may need to modify this based on your specific data structure
        return price_series.index[-169]  # Assuming last 168 hours are forecasted
    
    
    def run_experiment(self, start_date: str, num_days: int = 30, 
                      training_window_days: int = 73, refit_frequency: int = 7) -> Dict:
        """
        Run the rolling forecast experiment with sliding training window
        
        Args:
            start_date: Start date for the experiment (YYYY-MM-DD)
            num_days: Number of days to forecast
            training_window_days: Length of training window in days (default: 73 days)
            refit_frequency: Refit models every N days (default: 7 days)
        
        Returns:
            Dictionary with experiment results
        """
        start_dt = pd.to_datetime(start_date, utc=True)
        
        print(f"🚀 Starting {num_days}-day rolling forecast experiment from {start_date}")
        print(f"📊 Training window: {training_window_days} days (sliding window)")
        print(f"🔄 Model refit frequency: Every {refit_frequency} days")
        print("=" * 60)
        
        # Store fitted models for reuse
        current_sarima_model = None
        current_sarimax_model = None
        last_refit_day = -1
        
        for day in range(num_days):
            forecast_date = start_dt + timedelta(days=day)
            
            print(f"📅 Day {day + 1}/{num_days}: Forecasting for {forecast_date.strftime('%Y-%m-%d')}")
            
            # Calculate sliding training window
            train_start = start_dt - timedelta(days=training_window_days) + timedelta(days=day)
            train_end = forecast_date - timedelta(hours=1)
            
            print(f"   🧠 Training window: {train_start.strftime('%Y-%m-%d')} to {train_end.strftime('%Y-%m-%d %H:%M')}")
            
            # Get training data for this specific window
            training_subset = self.data.loc[train_start:train_end]
            
            if len(training_subset) < 168:
                print(f"   ⚠️ Insufficient training data ({len(training_subset)} hours), skipping...")
                continue
            
            # Check if we need to refit models
            should_refit = (day % refit_frequency == 0) or (current_sarima_model is None)
            
            if should_refit:
                print(f"   🔄 Refitting models (day {day + 1})...")
                last_refit_day = day
            else:
                print(f"   ♻️ Reusing models from day {last_refit_day + 1}")
            
            # Get actual value for forecast date (midnight)
            forecast_timestamp = forecast_date.replace(hour=0, minute=0, second=0, microsecond=0)
            
            if forecast_timestamp not in self.data.index:
                print(f"   ⚠️ No actual data available for {forecast_timestamp}, skipping...")
                continue
                
            actual_price = self.data.loc[forecast_timestamp, 'Price']
            
            if pd.isna(actual_price):
                print(f"   ⚠️ Actual price is NaN for {forecast_timestamp}, skipping...")
                continue
            
            print(f"   📊 Training data: {len(training_subset)} hours")
            
            # Generate forecasts (with conditional refitting)
            forecasts = self._generate_forecasts_optimized(
                training_subset, forecast_timestamp, should_refit,
                current_sarima_model, current_sarimax_model
            )
            
            # Update stored models if we refitted
            if should_refit:
                current_sarima_model = forecasts.get('sarima_model')
                current_sarimax_model = forecasts.get('sarimax_model')
            
            # Calculate RMSEs
            rmse_naive = calculate_rmse(actual_price, forecasts['naive'])
            rmse_sarima = calculate_rmse(actual_price, forecasts['sarima'])
            rmse_sarimax = calculate_rmse(actual_price, forecasts['sarimax'])
            
            # Store results
            self.results['dates'].append(forecast_timestamp)
            self.results['actual_prices'].append(actual_price)
            self.results['naive_forecasts'].append(forecasts['naive'])
            self.results['sarima_forecasts'].append(forecasts['sarima'])
            self.results['sarimax_forecasts'].append(forecasts['sarimax'])
            self.results['naive_rmse'].append(rmse_naive)
            self.results['sarima_rmse'].append(rmse_sarima)
            self.results['sarimax_rmse'].append(rmse_sarimax)
            
            # Print daily results
            print(f"   💰 Actual: {actual_price:.4f}")
            print(f"   🔮 Naive: {forecasts['naive']:.4f} (RMSE: {rmse_naive:.4f})")
            print(f"   📊 SARIMA: {forecasts['sarima']:.4f} (RMSE: {rmse_sarima:.4f})")
            print(f"   🎯 SARIMAX: {forecasts['sarimax']:.4f} (RMSE: {rmse_sarimax:.4f})")
            print("-" * 40)
        
        print("✅ Rolling forecast experiment completed!")
        return self._calculate_summary_stats()
    
    
    def _generate_forecasts_optimized(self, training_data: pd.DataFrame, 
                                     forecast_timestamp: pd.Timestamp, should_refit: bool,
                                     current_sarima_model: Any = None,
                                     current_sarimax_model: Any = None) -> Dict[str, float]:
        """
        Generate forecasts with conditional model refitting for optimization
        """
        forecasts = {'naive': np.nan, 'sarima': np.nan, 'sarimax': np.nan,
                    'sarima_model': None, 'sarimax_model': None}
        
        price_series = training_data['Price']
        
        # 1. Naive forecast (always fast)
        forecasts['naive'] = naive_forecast(price_series)
        
        # 2. SARIMA forecast (with conditional refitting)
        if should_refit or current_sarima_model is None:
            print(f"     🔄 Fitting new SARIMA model...")
            sarima_model = fit_sarima_model(price_series, self.sarima_order, self.seasonal_order)
            forecasts['sarima_model'] = sarima_model
        else:
            print(f"     ♻️ Using existing SARIMA model...")
            sarima_model = current_sarima_model
        
        if sarima_model is not None:
            try:
                # For reused models, we need to update with new data
                if not should_refit and current_sarima_model is not None:
                    # Use the existing model for forecast (simplified approach)
                    # Note: This is a simplified approach. For full accuracy, you'd want to 
                    # extend the model with new data, but that's complex with statsmodels
                    sarima_pred = sarima_model.forecast(steps=1)
                else:
                    sarima_pred = sarima_model.forecast(steps=1)
                
                forecasts['sarima'] = sarima_pred.iloc[0] if hasattr(sarima_pred, 'iloc') else sarima_pred[0]
                if should_refit:
                    self.fitted_models['sarima'].append(sarima_model)
            except Exception as e:
                print(f"     ⚠️ SARIMA forecast failed: {str(e)[:50]}...")
        
        # 3. SARIMAX forecast (with conditional refitting)
        if self.exog_variables:
            try:
                exog_train = training_data[self.exog_variables]
                
                # Get exogenous data for forecast point
                if forecast_timestamp in self.data.index:
                    exog_forecast = self.data.loc[[forecast_timestamp], self.exog_variables]
                    
                    if should_refit or current_sarimax_model is None:
                        print(f"     🔄 Fitting new SARIMAX model...")
                        sarimax_model = fit_sarimax_model(price_series, exog_train, 
                                                        self.sarima_order, self.seasonal_order)
                        forecasts['sarimax_model'] = sarimax_model
                    else:
                        print(f"     ♻️ Using existing SARIMAX model...")
                        sarimax_model = current_sarimax_model
                    
                    if sarimax_model is not None:
                        # Similar limitation as SARIMA - simplified forecast for reused models
                        sarimax_pred = sarimax_model.forecast(steps=1, exog=exog_forecast)
                        forecasts['sarimax'] = sarimax_pred.iloc[0] if hasattr(sarimax_pred, 'iloc') else sarimax_pred[0]
                        if should_refit:
                            self.fitted_models['sarimax'].append(sarimax_model)
                        
            except Exception as e:
                print(f"     ⚠️ SARIMAX forecast failed: {str(e)[:50]}...")
        
        return forecasts
    
    
    def _calculate_summary_stats(self) -> Dict:
        """
        Calculate summary statistics for the experiment
        """
        summary = {}
        
        for model in ['naive', 'sarima', 'sarimax']:
            rmse_values = [x for x in self.results[f'{model}_rmse'] if not pd.isna(x)]
            
            if rmse_values:
                summary[model] = {
                    'mean_rmse': np.mean(rmse_values),
                    'std_rmse': np.std(rmse_values),
                    'min_rmse': np.min(rmse_values),
                    'max_rmse': np.max(rmse_values),
                    'successful_forecasts': len(rmse_values)
                }
            else:
                summary[model] = {
                    'mean_rmse': np.nan,
                    'std_rmse': np.nan,
                    'min_rmse': np.nan,
                    'max_rmse': np.nan,
                    'successful_forecasts': 0
                }
        
        return summary
    
    
    def get_results_dataframe(self) -> pd.DataFrame:
        """
        Return results as a pandas DataFrame
        """
        return pd.DataFrame(self.results)

print("✅ RollingForecastExperiment class defined successfully!")

In [ ]:
# Cell 4: Configuration and Data Preparation

# =============================================================================
# EXPERIMENT CONFIGURATION
# =============================================================================
import time
# Model parameters
SARIMA_ORDER = (1, 1, 1)  # (p, d, q) - adjust based on your data characteristics
SEASONAL_ORDER = (1, 1, 1, 24)  # (P, D, Q, s) - 24 for hourly seasonality

# Experiment parameters
EXPERIMENT_START_DATE = "2025-03-15"  # Should match the RUN_DATE from Cell 0
NUM_FORECAST_DAYS = 30

# Exogenous variables (based on your build_training_set function output)
EXOG_VARIABLES = [
    'Load',
    'shortwave_radiation', 
    'temperature_2m',
    'direct_normal_irradiance',
    'diffuse_radiation',
    'Flow_NO',
    'yearday_cos',
    'Flow_GB',
    'month',
    'is_dst',
    'yearday_sin',
    'wind_speed_10m',
    'is_non_working_day',
    'hour_cos',
    'is_weekend',
    'cloud_cover',
    'weekday_sin',
    'hour_sin',
    'weekday_cos'
]

print("🔧 EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"📅 Start Date: {EXPERIMENT_START_DATE}")
print(f"⏰ Forecast Days: {NUM_FORECAST_DAYS}")
print(f"📊 SARIMA Order: {SARIMA_ORDER}")
print(f"🔄 Seasonal Order: {SEASONAL_ORDER}")
print(f"📈 Exogenous Variables: {len(EXOG_VARIABLES)}")
print("\n📋 Exogenous Variables List:")
for i, var in enumerate(EXOG_VARIABLES, 1):
    print(f"   {i}. {var}")

# =============================================================================
# DATA VALIDATION
# =============================================================================

print("\n🔍 VALIDATING TRAINING DATA")
print("=" * 50)

# Check if training_data exists
try:
    print(f"📊 Dataset shape: {training_data.shape}")
    print(f"📅 Date range: {training_data.index.min()} to {training_data.index.max()}")
    print(f"💰 Price column stats:")
    print(f"   - Mean: {training_data['Price'].mean():.2f}")
    print(f"   - Std: {training_data['Price'].std():.2f}")
    print(f"   - Min: {training_data['Price'].min():.2f}")
    print(f"   - Max: {training_data['Price'].max():.2f}")
    
    # Validate data structure
    is_valid = validate_training_data(training_data)
    
    if is_valid:
        # Check availability of exogenous variables
        available_exog = [var for var in EXOG_VARIABLES if var in training_data.columns]
        missing_exog = [var for var in EXOG_VARIABLES if var not in training_data.columns]
        
        if missing_exog:
            print(f"⚠️ Missing exogenous variables: {missing_exog}")
            print(f"✅ Available exogenous variables: {available_exog}")
            EXOG_VARIABLES = available_exog  # Update to only use available variables
            print(f"🔄 Updated exogenous variables list to {len(EXOG_VARIABLES)} variables")
        
        # Check data coverage for experiment period
        experiment_start = pd.to_datetime(EXPERIMENT_START_DATE, utc=True)
        experiment_end = experiment_start + timedelta(days=NUM_FORECAST_DAYS)
        
        data_start = training_data.index.min()
        data_end = training_data.index.max()
        
        # Ensure all timestamps are timezone-aware for comparison
        if data_start.tz is None:
            data_start = data_start.tz_localize('UTC')
        if data_end.tz is None:
            data_end = data_end.tz_localize('UTC')
            
        print(f"\n📅 Data Coverage Check:")
        print(f"   - Experiment needs: {experiment_start} to {experiment_end}")
        print(f"   - Data available: {data_start} to {data_end}")
        
        if experiment_start < data_start:
            print("❌ Experiment start date is before available data")
        elif experiment_end > data_end:
            print("⚠️ Experiment end date extends beyond available data")
            print("   This is expected if you have forecasted exogenous variables")
        else:
            print("✅ Data coverage is sufficient for the experiment")
        
        print("\n✅ Data validation completed successfully!")
        
    else:
        print("❌ Data validation failed. Please check your training_data DataFrame.")
        
except NameError:
    print("❌ training_data DataFrame not found!")
    print("Please ensure your training_data DataFrame is loaded before running this cell.")
except Exception as e:
    print(f"❌ Error during data validation: {str(e)}")

print("\n🚀 Ready to run the rolling forecast experiment!")

In [ ]:
# Cell 5 - Final Corrected 7-Day Rolling Forecast
# Clean version with all 19 exogenous variables and proper 7-day horizons

import time
from datetime import timedelta
import pandas as pd
import numpy as np

print("🎯 FINAL CORRECTED 7-DAY ROLLING FORECAST EXPERIMENT")
print("=" * 70)

# =============================================================================
# FINAL CORRECTED CONFIGURATION
# =============================================================================

# Fixed training window (doesn't shrink)
FIXED_TRAINING_WINDOW_DAYS = 73  # Always 73 days of training data
FORECAST_HORIZON_DAYS = 7  # 7-day ahead forecasting
REFIT_FREQUENCY = 7  # Weekly refits

# Keep your original model complexity if you want
FINAL_SARIMA_ORDER = (1, 1, 1)  # Your original SARIMA order
FINAL_SEASONAL_ORDER = (1, 1, 1, 24)  # Your original seasonal order

# KEEP ALL YOUR 19 EXOGENOUS VARIABLES - exactly as you had them!
FINAL_EXOG_VARIABLES = [
    'Load',
    'shortwave_radiation', 
    'temperature_2m',
    'direct_normal_irradiance',
    'diffuse_radiation',
    'Flow_NO',
    'yearday_cos',
    'Flow_GB',
    'month',
    'is_dst',
    'yearday_sin',
    'wind_speed_10m',
    'is_non_working_day',
    'hour_cos',
    'is_weekend',
    'cloud_cover',
    'weekday_sin',
    'hour_sin',
    'weekday_cos'
]

# Filter to available variables
available_final_exog = [var for var in FINAL_EXOG_VARIABLES if var in training_data.columns]

print(f"🔧 Final Configuration:")
print(f"   - Fixed training window: {FIXED_TRAINING_WINDOW_DAYS} days")
print(f"   - Forecast horizon: {FORECAST_HORIZON_DAYS} days (168 hours)")
print(f"   - SARIMA order: {FINAL_SARIMA_ORDER}")
print(f"   - Seasonal order: {FINAL_SEASONAL_ORDER}")
print(f"   - ALL Exogenous vars: {len(available_final_exog)}/{len(FINAL_EXOG_VARIABLES)}")
print(f"   - Variables: {available_final_exog}")

# =============================================================================
# SIMPLE WORKING EXPERIMENT FUNCTION
# =============================================================================

def run_7day_rolling_experiment(data, start_date, num_experiments=4):
    """
    Simple, working 7-day rolling forecast experiment
    """
    start_dt = pd.to_datetime(start_date, utc=True)
    
    results = {
        'experiment': [], 'origin_date': [], 'forecast_dates': [],
        'actual_prices': [], 'naive_forecasts': [], 'sarima_forecasts': [], 'sarimax_forecasts': [],
        'naive_rmse_7day': [], 'sarima_rmse_7day': [], 'sarimax_rmse_7day': []
    }
    
    print(f"\\n🎯 Running {num_experiments} × 7-day forecast experiments")
    print("=" * 60)
    
    for exp in range(num_experiments):
        # Forecast origin shifts by 7 days each time
        origin = start_dt + timedelta(days=exp * 7)
        
        print(f"\\n📅 Experiment {exp + 1}/{num_experiments}")
        print(f"   🎯 Origin: {origin.strftime('%Y-%m-%d')}")
        
        # FIXED training window - always same size
        train_start = origin - timedelta(days=FIXED_TRAINING_WINDOW_DAYS)
        train_end = origin - timedelta(hours=1)
        train_data = data.loc[train_start:train_end]
        
        print(f"   🧠 Training: {train_start.strftime('%Y-%m-%d')} to {train_end.strftime('%Y-%m-%d')} ({len(train_data)}h)")
        
        if len(train_data) < 168:
            print(f"   ❌ Insufficient training data, skipping")
            continue
        
        # Generate 7-day forecast dates
        forecast_dates = [origin + timedelta(days=d) for d in range(FORECAST_HORIZON_DAYS)]
        forecast_timestamps = [dt.replace(hour=0, minute=0, second=0, microsecond=0) for dt in forecast_dates]
        
        print(f"   📈 Forecasting: {forecast_dates[0].strftime('%m-%d')} to {forecast_dates[-1].strftime('%m-%d')}")
        
        # Get actual prices for 7 days
        actual_prices = []
        for ts in forecast_timestamps:
            if ts in data.index and not pd.isna(data.loc[ts, 'Price']):
                actual_prices.append(float(data.loc[ts, 'Price']))
            else:
                actual_prices.append(np.nan)
        
        valid_actuals = [p for p in actual_prices if not pd.isna(p)]
        print(f"   💰 Actual prices available: {len(valid_actuals)}/7 days")
        
        # 1. NAIVE FORECAST - repeat last price
        price_series = train_data['Price'].astype('float64').dropna()
        last_price = float(price_series.iloc[-1])
        naive_forecasts = [last_price] * 7
        
        # 2. SARIMA FORECAST
        sarima_forecasts = [np.nan] * 7
        try:
            if len(price_series) >= 48:
                sarima_model = SARIMAX(price_series, 
                                     order=FINAL_SARIMA_ORDER, 
                                     seasonal_order=FINAL_SEASONAL_ORDER,
                                     enforce_stationarity=False, 
                                     enforce_invertibility=False)
                fitted_sarima = sarima_model.fit(disp=False, maxiter=100, method='lbfgs')
                sarima_pred = fitted_sarima.forecast(steps=7)
                sarima_forecasts = [float(x) for x in sarima_pred]
                print(f"   ✅ SARIMA fitted successfully")
        except Exception as e:
            print(f"   ⚠️ SARIMA failed: {str(e)[:50]}...")
            sarima_forecasts = naive_forecasts.copy()
        
        # 3. SARIMAX FORECAST with ALL YOUR FEATURES
        sarimax_forecasts = [np.nan] * 7
        try:
            if available_final_exog and len(price_series) >= 48:
                # Get exogenous training data
                exog_train = train_data[available_final_exog].astype('float64')
                
                # Get exogenous forecast data for 7 days
                exog_forecast_list = []
                for ts in forecast_timestamps:
                    if ts in data.index:
                        exog_forecast_list.append(data.loc[ts, available_final_exog].astype('float64'))
                    else:
                        # Use last available values
                        exog_forecast_list.append(exog_train.iloc[-1])
                
                exog_forecast = pd.DataFrame(exog_forecast_list, columns=available_final_exog)
                
                # Align data and remove NaN
                combined = pd.concat([price_series, exog_train], axis=1).dropna()
                
                if len(combined) >= 48:
                    clean_price = combined.iloc[:, 0]
                    clean_exog = combined.iloc[:, 1:]
                    
                    sarimax_model = SARIMAX(clean_price, 
                                          exog=clean_exog,
                                          order=FINAL_SARIMA_ORDER,
                                          seasonal_order=FINAL_SEASONAL_ORDER,
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
                    fitted_sarimax = sarimax_model.fit(disp=False, maxiter=100, method='lbfgs')
                    sarimax_pred = fitted_sarimax.forecast(steps=7, exog=exog_forecast)
                    sarimax_forecasts = [float(x) for x in sarimax_pred]
                    print(f"   ✅ SARIMAX fitted with {len(available_final_exog)} variables")
        except Exception as e:
            print(f"   ⚠️ SARIMAX failed: {str(e)[:50]}...")
            sarimax_forecasts = naive_forecasts.copy()
        
        # Calculate 7-day RMSE for each model
        def calc_7day_rmse(actual, forecast):
            valid_pairs = [(a, f) for a, f in zip(actual, forecast) if not pd.isna(a) and not pd.isna(f)]
            if len(valid_pairs) == 0:
                return np.nan
            mse = np.mean([(a - f) ** 2 for a, f in valid_pairs])
            return np.sqrt(mse)
        
        rmse_naive = calc_7day_rmse(actual_prices, naive_forecasts)
        rmse_sarima = calc_7day_rmse(actual_prices, sarima_forecasts)
        rmse_sarimax = calc_7day_rmse(actual_prices, sarimax_forecasts)
        
        # Store results
        results['experiment'].append(exp + 1)
        results['origin_date'].append(origin)
        results['forecast_dates'].append(forecast_timestamps)
        results['actual_prices'].append(actual_prices)
        results['naive_forecasts'].append(naive_forecasts)
        results['sarima_forecasts'].append(sarima_forecasts)
        results['sarimax_forecasts'].append(sarimax_forecasts)
        results['naive_rmse_7day'].append(rmse_naive)
        results['sarima_rmse_7day'].append(rmse_sarima)
        results['sarimax_rmse_7day'].append(rmse_sarimax)
        
        # Display results
        print(f"   📊 7-Day RMSE Results:")
        print(f"      🎯 Naive:   {rmse_naive:.6f}")
        print(f"      📈 SARIMA:  {rmse_sarima:.6f}")
        print(f"      🎯 SARIMAX: {rmse_sarimax:.6f}")
        
        if len(valid_actuals) > 0:
            print(f"   📋 Day 1 Comparison:")
            print(f"      Actual: {actual_prices[0]:.4f} | Naive: {naive_forecasts[0]:.4f} | SARIMA: {sarima_forecasts[0]:.4f} | SARIMAX: {sarimax_forecasts[0]:.4f}")
        
        print("-" * 60)
    
    return results

# =============================================================================
# RUN THE FINAL EXPERIMENT
# =============================================================================

print(f"\\n🚀 EXECUTING FINAL EXPERIMENT")
print("=" * 60)

start_time = time.time()

try:
    # Run the corrected experiment
    final_results = run_7day_rolling_experiment(
        data=training_data,
        start_date=EXPERIMENT_START_DATE,  # 2025-03-15
        num_experiments=4  # 4 × 7-day experiments
    )
    
    end_time = time.time()
    print(f"\\n⏱️ FINAL experiment completed in {end_time - start_time:.1f} seconds!")
    
    # Calculate summary statistics
    summary_final = {}
    for model in ['naive', 'sarima', 'sarimax']:
        rmse_values = [x for x in final_results[f'{model}_rmse_7day'] if not pd.isna(x)]
        if rmse_values:
            summary_final[model] = {
                'mean_rmse_7day': np.mean(rmse_values),
                'std_rmse_7day': np.std(rmse_values),
                'successful_experiments': len(rmse_values)
            }
        else:
            summary_final[model] = {
                'mean_rmse_7day': np.nan, 
                'std_rmse_7day': np.nan, 
                'successful_experiments': 0
            }
    
    # Display final results
    print(f"\\n📊 FINAL EXPERIMENT RESULTS (7-Day RMSE)")
    print("=" * 60)
    for model_name, stats in summary_final.items():
        print(f"🔹 {model_name.upper()}:")
        print(f"   Mean 7-Day RMSE: {stats['mean_rmse_7day']:.6f}")
        print(f"   Std 7-Day RMSE:  {stats['std_rmse_7day']:.6f}")
        print(f"   Successful Experiments: {stats['successful_experiments']}/4")
    
    # Create results DataFrame
    final_results_df = pd.DataFrame(final_results)
    
    print(f"\\n✅ Final results available in 'final_results_df'")
    print(f"✅ Final summary in 'summary_final'")
    
    # Show which model performed best
    valid_models = {name: stats for name, stats in summary_final.items() 
                   if not pd.isna(stats['mean_rmse_7day'])}
    
    if valid_models:
        best_model = min(valid_models.keys(), key=lambda x: valid_models[x]['mean_rmse_7day'])
        best_rmse = valid_models[best_model]['mean_rmse_7day']
        print(f"\\n🏆 BEST MODEL: {best_model.upper()} (RMSE: {best_rmse:.6f})")

except Exception as e:
    print(f"❌ Final experiment failed: {e}")
    import traceback
    traceback.print_exc()

print(f"\\n💡 Final experiment features:")
print(f"   ✅ Fixed 73-day training windows")
print(f"   ✅ True 7-day forecast horizons") 
print(f"   ✅ ALL {len(available_final_exog)} of your exogenous variables")
print(f"   ✅ Your original SARIMA({FINAL_SARIMA_ORDER[0]},{FINAL_SARIMA_ORDER[1]},{FINAL_SARIMA_ORDER[2]}) + seasonal({FINAL_SEASONAL_ORDER[0]},{FINAL_SEASONAL_ORDER[1]},{FINAL_SEASONAL_ORDER[2]},{FINAL_SEASONAL_ORDER[3]}) specification")
print(f"   ✅ Proper 7-day RMSE calculation")
print(f"   ✅ 4 rolling experiments covering 28 days")

   ✅ SARIMA fitted successfully
   ✅ SARIMAX fitted with 19 variables
   📊 7-Day RMSE Results:
      🎯 Naive:   0.008536
      📈 SARIMA:  0.008243
      🎯 SARIMAX: 0.036659
   📋 Day 1 Comparison:
      Actual: 0.1121 | Naive: 0.1052 | SARIMA: 0.1009 | SARIMAX: 0.0950
------------------------------------------------------------
\n📅 Experiment 2/4
   🎯 Origin: 2025-03-22
   🧠 Training: 2025-01-08 to 2025-03-21 (1752h)
   📈 Forecasting: 03-22 to 03-28
   💰 Actual prices available: 1/7 days
   ✅ SARIMA fitted successfully


In [ ]:
# Cell 6: Results Analysis and Visualization

# =============================================================================
# DETAILED RESULTS ANALYSIS
# =============================================================================

print("📊 DETAILED RESULTS ANALYSIS")
print("=" * 60)

try:
    # Display results dataframe summary
    print(f"📋 Results DataFrame Shape: {results_df.shape}")
    print(f"📅 Date Range: {results_df['dates'].min()} to {results_df['dates'].max()}")
    
    # Calculate additional metrics
    valid_results = results_df.dropna(subset=['actual_prices'])
    
    if len(valid_results) > 0:
        print(f"\n✅ Valid forecasts: {len(valid_results)}/{len(results_df)}")
        
        # Price statistics
        print(f"\n💰 ACTUAL PRICE STATISTICS:")
        print(f"   Mean: {valid_results['actual_prices'].mean():.2f}")
        print(f"   Std:  {valid_results['actual_prices'].std():.2f}")
        print(f"   Min:  {valid_results['actual_prices'].min():.2f}")
        print(f"   Max:  {valid_results['actual_prices'].max():.2f}")
        
        # Model comparison table
        print(f"\n📊 MODEL COMPARISON TABLE:")
        print("-" * 80)
        print(f"{'Model':<12} {'Mean RMSE':<12} {'Std RMSE':<12} {'Min RMSE':<12} {'Max RMSE':<12}")
        print("-" * 80)
        
        for model in ['naive', 'sarima', 'sarimax']:
            if model in summary_stats:
                stats = summary_stats[model]
                print(f"{model.capitalize():<12} "
                      f"{stats['mean_rmse']:<12.4f} "
                      f"{stats['std_rmse']:<12.4f} "
                      f"{stats['min_rmse']:<12.4f} "
                      f"{stats['max_rmse']:<12.4f}")
        print("-" * 80)

except NameError:
    print("❌ Results not available. Please run the experiment first (Cell 5).")
    print("🔄 Attempting to create sample visualization with dummy data...")

# =============================================================================
# VISUALIZATIONS
# =============================================================================

print("\n📈 CREATING VISUALIZATIONS")
print("=" * 60)

try:
    # Create a comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('30-Day Rolling Forecast Experiment Results', fontsize=16, fontweight='bold')
    
    # 1. RMSE Evolution Over Time
    ax1 = axes[0, 0]
    valid_results = results_df.dropna(subset=['dates'])
    
    if len(valid_results) > 0:
        ax1.plot(valid_results['dates'], valid_results['naive_rmse'], 
                marker='o', label='Naive', linewidth=2, markersize=4)
        ax1.plot(valid_results['dates'], valid_results['sarima_rmse'], 
                marker='s', label='SARIMA', linewidth=2, markersize=4)
        ax1.plot(valid_results['dates'], valid_results['sarimax_rmse'], 
                marker='^', label='SARIMAX', linewidth=2, markersize=4)
        
        ax1.set_title('RMSE Evolution Over 30 Days')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('RMSE')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
    
    # 2. Box Plot of RMSE Distribution
    ax2 = axes[0, 1]
    rmse_data = []
    labels = []
    
    for model in ['naive', 'sarima', 'sarimax']:
        rmse_col = f'{model}_rmse'
        if rmse_col in results_df.columns:
            model_rmse = results_df[rmse_col].dropna()
            if len(model_rmse) > 0:
                rmse_data.append(model_rmse)
                labels.append(model.capitalize())
    
    if rmse_data:
        ax2.boxplot(rmse_data, labels=labels)
        ax2.set_title('RMSE Distribution by Model')
        ax2.set_ylabel('RMSE')
        ax2.grid(True, alpha=0.3)
    
    # 3. Actual vs Predicted Scatter Plot
    ax3 = axes[1, 0]
    if len(valid_results) > 0:
        # SARIMAX scatter (best model typically)
        valid_pred = valid_results.dropna(subset=['actual_prices', 'sarimax_forecasts'])
        if len(valid_pred) > 0:
            ax3.scatter(valid_pred['actual_prices'], valid_pred['sarimax_forecasts'], 
                       alpha=0.6, s=50, label='SARIMAX')
            
        # Perfect prediction line
        if len(valid_results) > 0:
            min_price = valid_results['actual_prices'].min()
            max_price = valid_results['actual_prices'].max()
            ax3.plot([min_price, max_price], [min_price, max_price], 
                    'r--', alpha=0.8, label='Perfect Prediction')
        
        ax3.set_title('Actual vs Predicted Prices (SARIMAX)')
        ax3.set_xlabel('Actual Price')
        ax3.set_ylabel('Predicted Price')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Model Performance Summary Bar Chart
    ax4 = axes[1, 1]
    models = []
    mean_rmses = []
    
    for model in ['naive', 'sarima', 'sarimax']:
        if model in summary_stats and not pd.isna(summary_stats[model]['mean_rmse']):
            models.append(model.capitalize())
            mean_rmses.append(summary_stats[model]['mean_rmse'])
    
    if models and mean_rmses:
        bars = ax4.bar(models, mean_rmses, 
                      color=['skyblue', 'lightcoral', 'lightgreen'])
        ax4.set_title('Mean RMSE by Model')
        ax4.set_ylabel('Mean RMSE')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar, value in zip(bars, mean_rmses):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(mean_rmses)*0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualization completed successfully!")

except Exception as e:
    print(f"⚠️ Visualization error: {str(e)}")
    print("🔧 This might be due to insufficient data or missing results")

# =============================================================================
# FEATURE IMPORTANCE ANALYSIS (SARIMAX)
# =============================================================================

print(f"\n🎯 SARIMAX FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

try:
    if hasattr(experiment, 'fitted_models') and experiment.fitted_models['sarimax']:
        # Get the last fitted SARIMAX model for analysis
        last_sarimax_model = experiment.fitted_models['sarimax'][-1]
        
        if hasattr(last_sarimax_model, 'params') and len(EXOG_VARIABLES) > 0:
            print("📊 EXOGENOUS VARIABLE COEFFICIENTS (Last Model):")
            print("-" * 50)
            
            # Extract coefficients for exogenous variables
            params = last_sarimax_model.params
            
            # The exogenous coefficients typically come after the ARIMA parameters
            # This is a simplified approach - you might need to adjust based on your model structure
            if len(params) > len(SARIMA_ORDER) + len(SEASONAL_ORDER):
                exog_start_idx = len(SARIMA_ORDER) + len(SEASONAL_ORDER) - 2  # Approximate
                
                for i, var_name in enumerate(EXOG_VARIABLES):
                    if exog_start_idx + i < len(params):
                        coef_value = params.iloc[exog_start_idx + i]
                        print(f"   {var_name:<20}: {coef_value:>8.4f}")
            
            print("\n💡 Note: Coefficient interpretation depends on model specification")
            print("   Larger absolute values indicate stronger influence on price forecasts")
        
        else:
            print("⚠️ Feature importance analysis not available")
            print("   Model parameters not accessible or no exogenous variables")
    
    else:
        print("⚠️ No SARIMAX models available for feature importance analysis")

except Exception as e:
    print(f"⚠️ Feature importance analysis failed: {str(e)}")
    print("🔧 This is normal if SARIMAX models failed to fit")

print(f"\n📋 RESULTS SUMMARY SAVED")
print("=" * 60)
print("✅ All analysis completed!")
print("📊 Key variables available for further analysis:")
print("   - results_df: Detailed daily results")
print("   - summary_stats: Model performance summary")
print("   - experiment: Full experiment object with fitted models")

In [ ]:
# Cell 7: Model Recommendation and Insights

# =============================================================================
# MODEL RECOMMENDATION ENGINE
# =============================================================================

print("🎯 MODEL RECOMMENDATION AND INSIGHTS")
print("=" * 60)

def get_best_model_recommendation(summary_stats: Dict) -> Dict:
    """
    Determine the best performing model based on multiple criteria
    """
    recommendation = {
        'best_model': None,
        'best_rmse': float('inf'),
        'criteria': {},
        'insights': []
    }
    
    valid_models = {}
    
    # Filter out models with invalid results
    for model_name, stats in summary_stats.items():
        if not pd.isna(stats['mean_rmse']) and stats['successful_forecasts'] > 0:
            valid_models[model_name] = stats
    
    if not valid_models:
        recommendation['insights'].append("❌ No valid model results found")
        return recommendation
    
    # 1. Best Mean RMSE
    best_rmse_model = min(valid_models.keys(), 
                         key=lambda x: valid_models[x]['mean_rmse'])
    recommendation['best_model'] = best_rmse_model
    recommendation['best_rmse'] = valid_models[best_rmse_model]['mean_rmse']
    
    # 2. Most Consistent (Lowest Std RMSE)
    most_consistent_model = min(valid_models.keys(), 
                               key=lambda x: valid_models[x]['std_rmse'])
    
    # 3. Most Reliable (Highest Success Rate)
    most_reliable_model = max(valid_models.keys(), 
                             key=lambda x: valid_models[x]['successful_forecasts'])
    
    # Store criteria results
    recommendation['criteria'] = {
        'best_accuracy': best_rmse_model,
        'most_consistent': most_consistent_model,
        'most_reliable': most_reliable_model
    }
    
    # Generate insights
    recommendation['insights'].append(
        f"🏆 Best Overall Model: {best_rmse_model.upper()} "
        f"(Mean RMSE: {recommendation['best_rmse']:.4f})"
    )
    
    if most_consistent_model != best_rmse_model:
        recommendation['insights'].append(
            f"📊 Most Consistent: {most_consistent_model.upper()} "
            f"(Std RMSE: {valid_models[most_consistent_model]['std_rmse']:.4f})"
        )
    
    if most_reliable_model != best_rmse_model:
        recommendation['insights'].append(
            f"🔧 Most Reliable: {most_reliable_model.upper()} "
            f"({valid_models[most_reliable_model]['successful_forecasts']}/{NUM_FORECAST_DAYS} forecasts)"
        )
    
    # Performance comparison insights
    rmse_values = [stats['mean_rmse'] for stats in valid_models.values()]
    rmse_improvement = (max(rmse_values) - min(rmse_values)) / max(rmse_values) * 100
    
    recommendation['insights'].append(
        f"📈 Performance Improvement: {rmse_improvement:.1f}% "
        f"(Best vs Worst Model)"
    )
    
    return recommendation

# =============================================================================
# GENERATE RECOMMENDATIONS
# =============================================================================

try:
    # Get model recommendation
    recommendation = get_best_model_recommendation(summary_stats)
    
    print("🏆 MODEL PERFORMANCE RANKING")
    print("-" * 40)
    
    # Rank models by mean RMSE
    valid_models = {name: stats for name, stats in summary_stats.items() 
                   if not pd.isna(stats['mean_rmse'])}
    
    if valid_models:
        sorted_models = sorted(valid_models.items(), 
                              key=lambda x: x[1]['mean_rmse'])
        
        for rank, (model_name, stats) in enumerate(sorted_models, 1):
            medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉"
            print(f"{medal} {rank}. {model_name.upper()}")
            print(f"     Mean RMSE: {stats['mean_rmse']:.4f}")
            print(f"     Consistency: {stats['std_rmse']:.4f}")
            print(f"     Success Rate: {stats['successful_forecasts']}/{NUM_FORECAST_DAYS}")
            print()
    
    print("💡 KEY INSIGHTS")
    print("-" * 40)
    for insight in recommendation['insights']:
        print(f"   {insight}")
    
    print(f"\n🎯 FINAL RECOMMENDATION")
    print("-" * 40)
    if recommendation['best_model']:
        print(f"✅ Recommended Model: {recommendation['best_model'].upper()}")
        print(f"📊 Expected RMSE: {recommendation['best_rmse']:.4f}")
        
        # Model-specific recommendations
        if recommendation['best_model'] == 'naive':
            print(f"\n🔍 NAIVE MODEL INSIGHTS:")
            print(f"   • Simple yesterday=today approach works surprisingly well")
            print(f"   • Consider this as a strong baseline for comparison")
            print(f"   • Low computational cost, high interpretability")
            
        elif recommendation['best_model'] == 'sarima':
            print(f"\n🔍 SARIMA MODEL INSIGHTS:")
            print(f"   • Pure time series approach captures patterns effectively")
            print(f"   • Model order: {SARIMA_ORDER} with seasonality {SEASONAL_ORDER}")
            print(f"   • Consider tuning hyperparameters for better performance")
            
        elif recommendation['best_model'] == 'sarimax':
            print(f"\n🔍 SARIMAX MODEL INSIGHTS:")
            print(f"   • Exogenous variables provide valuable additional information")
            print(f"   • Using {len(EXOG_VARIABLES)} external features")
            print(f"   • Most complex model - ensure robust exogenous forecasts")
        
        print(f"\n📋 IMPLEMENTATION CONSIDERATIONS:")
        print(f"   • Computational Cost: {'Low' if recommendation['best_model'] == 'naive' else 'Medium' if recommendation['best_model'] == 'sarima' else 'High'}")
        print(f"   • Data Requirements: {'Minimal' if recommendation['best_model'] == 'naive' else 'Historical Prices' if recommendation['best_model'] == 'sarima' else 'Prices + Exogenous'}")
        print(f"   • Interpretability: {'High' if recommendation['best_model'] == 'naive' else 'Medium'}")
        
    else:
        print("❌ No clear recommendation - all models failed")
        print("🔧 Consider:")
        print("   • Checking data quality and completeness")
        print("   • Adjusting model parameters")
        print("   • Using a longer training period")
    
except NameError:
    print("❌ Summary statistics not available")
    print("🔄 Please run the experiment first (Cell 5)")
except Exception as e:
    print(f"⚠️ Recommendation generation failed: {str(e)}")

# =============================================================================
# ADDITIONAL INSIGHTS AND NEXT STEPS
# =============================================================================

print(f"\n🚀 NEXT STEPS AND IMPROVEMENTS")
print("=" * 60)
print("🎨 Model Enhancement Ideas:")
print("   1. Hyperparameter Tuning:")
print("      • Grid search for optimal SARIMA orders")
print("      • Cross-validation for model selection")
print("   2. Feature Engineering:")
print("      • Lag features, rolling averages")
print("      • Calendar effects (holidays, seasons)")
print("   3. Advanced Models:")
print("      • Machine Learning approaches (XGBoost, LSTM)")
print("      • Ensemble methods combining multiple forecasts")
print("   4. Evaluation Improvements:")
print("      • Multi-step ahead forecasting")
print("      • Additional metrics (MAE, MAPE)")
print("      • Directional accuracy assessment")

print(f"\n📊 Experiment Variables Available:")
print("   • results_df: Daily forecast results")
print("   • summary_stats: Model performance statistics") 
print("   • experiment: Complete experiment object")
print("   • recommendation: Model recommendation analysis")

print(f"\n✅ 30-Day Rolling Forecast Experiment Complete!")
print("🎯 Use the insights above to improve your electricity price forecasting system.")